In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import numpy.fft as fourier
log2 = np.log2
log10 = np.log10
real = np.real
imag = np.imag 
pi = np.pi

def combined_analysis(s_df, dt, NfDec, max_seg_period):

    freqs_list = []
    ensemble_ave_list = []
    ensemble_errors_list = []
    
    freqs_binned_list = []
    PSD_binned_list = []
    dPSD_list = []

    Dphi_list = []
    freqs_lag_list = []
    taus_list = []
    dtaus_list = []
    gamma2_list = []

    # Call ensemble_averaging for each row in s_df
    for index, row in s_df.iterrows():
        s = row.values

        freqs, ensemble_ave, ensemble_errors = ensemble_averaging(s, dt, max_seg_period)

        freqs_list.append(freqs)
        ensemble_ave_list.append(ensemble_ave)
        ensemble_errors_list.append(ensemble_errors)

    # Call binned_spectrum for each row in s_df
    for index, row in s_df.iterrows():
        s = row.values
        freqs_binned, PSD_binned, dPSD = binned_spectrum(s, dt, NfDec, max_seg_period)
    
        freqs_binned_list.append(freqs_binned)
        PSD_binned_list.append(PSD_binned)
        dPSD_list.append(dPSD)

    # Call FourierLag for each row in s_df excluding the first row
    for index,row in s_df.iterrows():
        s = s_df.iloc[0].values
        h = row.values
        Dphi, freqs, taus, dtaus, gamma2 = FourierLag(s, h, dt, NfDec, max_seg_period)

        Dphi_list.append(Dphi)
        freqs_lag_list.append(freqs)
        taus_list.append(taus)
        dtaus_list.append(dtaus)
        
        gamma2_list.append(gamma2)

    # Create DataFrames from the collected results
    freqs_df = pd.DataFrame(freqs_list, index=s_df.index)
    ensemble_ave_df = pd.DataFrame(ensemble_ave_list, index=s_df.index)
    ensemble_errors_df = pd.DataFrame(ensemble_errors_list, index=s_df.index)
    
    freqs_binned_df = pd.DataFrame(freqs_binned_list, index=s_df.index)
    PSD_binned_df = pd.DataFrame(PSD_binned_list, index=s_df.index)
    dPSD_df = pd.DataFrame(dPSD_list, index=s_df.index)

    Dphi_df = pd.DataFrame(Dphi_list, index=s_df.index)
    freqs_lag_df = pd.DataFrame(freqs_lag_list, index=s_df.index)
    taus_df = pd.DataFrame(taus_list, index=s_df.index)
    dtaus_df = pd.DataFrame(dtaus_list, index=s_df.index)
    gamma2_df = pd.DataFrame(gamma2_list, index=s_df.index)

    return (freqs_df, ensemble_ave_df, ensemble_errors_df, freqs_binned_df, PSD_binned_df, dPSD_df, Dphi_df, 
            freqs_lag_df, taus_df, dtaus_df, gamma2_df)

def ensemble_averaging(s,dt, max_seg_period):
    '''This function takes an input curve, s, and a segment length, max_seg_period, (which has the same units as the time binning, dt - in my case, seconds).
    It first slices the input curve into M segments of length di, where di is the largest power of 2 such that di < max_seg_period/dt.
    Ensuring that the segments are a power of 2 in length allows the fast fourier transform to work efficiently.
    
    For each slice, it then computes the periodogram (raw PSD) which will have maximum frequency 1/(2*dt), and minimum frequency 1/(di*dt).
    In each frequency bin, it then takes the mean of the periodograms at this frequency, to yield the ensemble averaged PSD.
    This output is then ready for logarithmic rebinning.
    '''    
    di = 2 ** int(log2(max_seg_period/dt)) # Slice length.
    freqs = abs(fourier.fftfreq(di, dt))[1:int(di/2+1)] # Raw frequencies of the sliced data.

    M =  int(len(s)/di) # Number of slices.
    s = s[0: M*di] #  We discard the final few data points such that each segment has length di.
    s.shape = (M, di) # Reshape our data so that we can iterate over rows (segments).

    periodograms = np.zeros((M, int(di/2)), dtype = 'float') # Placeholder array.
 
    for j in range(M):
        S = fourier.fft(s[j]) / di # Fourier transform of segment j.
        raw_periodogram_j = (np.conj(S) * S) * di *2* dt / np.average(s[j])**2 # Raw periodogram of segment j.
        periodograms[j] = abs(raw_periodogram_j)[1:int(di/2+1)] #Take only the positive side of the raw periodogram as it is symmetric about zero and the lower half will change the norm incorrectly.
    ensemble_ave = periodograms.sum(axis = 0)/M # Average all of the periodograms for each frequency bin and voila.
    ensemble_errors = np.sqrt(np.var(periodograms, axis=0, ddof = 1)) / np.sqrt(M) # Compute the ensemble errors.

    return freqs, ensemble_ave, ensemble_errors

def binned_spectrum(s,dt, NfDec, max_seg_period):
    '''The function takes an input, s, runs it through the ensemble averaging function, and then rebins the resulting power spectrum logarithmically, greatly reducing the noise.
    The parameter NfDec determines the number of logarithmic bins per frequency decaed. It can be turned up if you want a noisier power spectrum with more features, or
    down for a less noisy power spectrum, although this may mask interesting features. Feel free to tune it until your output is pretty.
    max_seg_period is explained in the ensemble_averaging comments.'''
    
    A = ensemble_averaging(s,dt, max_seg_period = max_seg_period)
    freqs, ensemble_ave, ensemble_errors  = A[0], A[1], A[2] # First ensemble average the input curve to smooth out anomalous fourier features.

    fbins = np.logspace(log10(freqs[0]), log10(freqs[-1]), int(log10(freqs[-1]) - log10(freqs[0])) * NfDec) # We define the logarithmic bins for our output PSD.
    freqs_binned = np.array((), dtype = 'float') # Placeholders
    PSD_binned = np.array((), dtype = 'float')
    dPSD = np.array((), dtype = 'float')
    inds = np.digitize(freqs, fbins) # Assign frequencies to their parent bins and store the indices.
    
    count =0
    for i in range(1, inds[-1]):
        if i in inds:
            i_min, i_max = min(np.argwhere(inds == i))[0], max(np.argwhere(inds==i))[0] # Take the min and max index (in freqs) for each bin in fbins.
            if(i_min==i_max and count==0):
                count+=1
            elif(count > 0):
                freqs_binned = np.append(freqs_binned, (freqs[i_min-count] + freqs[i_max])/2) # The average frequency in each fbin will then be the bin centre.
                PSD_binned = np.append(PSD_binned, np.average(ensemble_ave[i_min-count:i_max+1])) # And the average PSD in each fbin will be... the average PSD in each fbin :P
                dPSD = np.append(dPSD, np.sqrt(sum(ensemble_errors[i_min-count:i_max+1]**2))/(i_max-i_min+count))
                count=0
            else:
                freqs_binned = np.append(freqs_binned, (freqs[i_min] + freqs[i_max])/2) # The average frequency in each fbin will then be the bin centre.
                PSD_binned = np.append(PSD_binned, np.average(ensemble_ave[i_min:i_max+1])) # And the average PSD in each fbin will be... the average PSD in each fbin :P
                dPSD = np.append(dPSD, np.sqrt(sum(ensemble_errors[i_min:i_max+1]**2))/(i_max-i_min)) # If the bin does not have size 1, then the errors are calculated as the quadratic sum of the ensemble errors in the bin.
            # else:
            #     dPSD = np.append(dPSD, ensemble_errors[i_min]) # If it has size 1, the fbin error is just the ensemble averaged error.
        else:
            pass
    
    return freqs_binned, PSD_binned, dPSD

def FourierLag(s, h, dt, NfDec, max_seg_period):
    
    '''Procedure to calculate the Fourier lag as in Uttley (2014), omitting background and poisson error terms as input from simulations.'''
    P_s = binned_spectrum(s, dt, NfDec = NfDec, max_seg_period = max_seg_period)[1] # Used later for error calcs.
    P_h = binned_spectrum(h, dt, NfDec = NfDec, max_seg_period = max_seg_period)[1]
    #print(P_s)
    
    di = 2 ** int(log2(max_seg_period/dt))
    M =  int(len(s)/di)
    freqs = abs(fourier.fftfreq(di, dt))[1:int(di/2+1)] # Raw frequencies of the sliced data.
    fbins = np.logspace(log10(freqs[0]), log10(freqs[-1]), int(log10(freqs[-1]) - log10(freqs[0])) * NfDec) # We define the logarithmic bins for our output PSD.
    # print len(freqs),len(fbins),len(freqs)/(1.0*len(fbins)),int(log10(freqs[-1]) - log10(freqs[0])) 
    # exit()
    #Here we slice up the 'soft' and 'hard' time series as in ensemble_averaging.
    h = h[0: M*di]
    h.shape = (M, di)
    s = s[0: M*di] 
    s.shape = (M, di)

    #Compute the ensemble averaged cross spectra.
    
    freqs_ens = abs(fourier.fftfreq(di, dt)[1:int(di/2+1)])  # Ensure this is outside the loop

    #Compute the ensemble averaged cross spectra
    cross_spec = np.zeros((M, int(di/2)), dtype='complex')
    for j in range(M):
        H = fourier.fft(h[j]) / di
        S = fourier.fft(s[j]) / di

        cross_0 = (np.conj(S) * H) * di * 2 * dt / (np.average(h[j]) * np.average(s[j]))
        cross_spec[j] = cross_0[1:int(di/2+1)]

    cross_ens = cross_spec.sum(axis=0) / M

    # Split into real and imaginary components
    cross_re_ens = real(cross_ens)
    cross_im_ens = imag(cross_ens)

    #Logarithmically bin the cross spectral components.
    freqs = np.array(())
    cross_re = np.array((), dtype = 'float')
    cross_im = np.array((), dtype = 'float')
    
    inds = np.digitize(freqs_ens, fbins)
    K = np.array(())

    #print('inds:', inds)
    #print('freqs_ens:', freqs_ens)
    
    count=0
    for i in range(1, inds[-1]):
        if i in inds:
            i_min, i_max = min(np.argwhere(inds == i))[0], max(np.argwhere(inds==i))[0]
            if(i_min==i_max and count==0):
                count+=1
            elif(count > 0):
                freqs = np.append(freqs, (freqs_ens[i_min-count] + freqs_ens[i_max])/2)
                cross_re = np.append(cross_re, np.average(cross_re_ens[i_min-count:i_max+1]))
                cross_im = np.append(cross_im, np.average(cross_im_ens[i_min-count:i_max+1]))
                K = np.append(K, i_max-i_min+count+1)
                count=0
            else:
                freqs = np.append(freqs, (freqs_ens[i_min] + freqs_ens[i_max])/2)
                cross_re = np.append(cross_re, np.average(cross_re_ens[i_min:i_max+1]))
                cross_im = np.append(cross_im, np.average(cross_im_ens[i_min:i_max+1]))
                K = np.append(K, i_max-i_min+1)           
        else:
            pass

    #From phase lags, Dphi, get time lags, taus.
    Dphi = np.arctan2(cross_im, cross_re) #NOT THE ABSOLUTE VALUE!!!
    taus = -Dphi / (2*pi*freqs)
    #print('P_s:', P_s, 'P_h:', P_h, 'cross_re:', cross_re, 'cross_im:', cross_im)
    #Compute errors on Dphi and taus.
    gamma2 = (cross_re**2 + cross_im**2) / (P_s* P_h)
    
    for l in range(len(gamma2)):
        if gamma2[l] < 0:
            gamma2[l]= 1 / (K[l]*M)
    
    d1phi = np.sqrt((1.-gamma2)/(2.*gamma2*K*M))
    dtaus = d1phi / (2.*pi*freqs)
    return Dphi,freqs, taus, dtaus, gamma2